# Geography of the Amazon IN 
Sale Report
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geopandas as gpd
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file into a Pandas DataFrame
# zipcode_df = pd.read_csv("../Amazon Sale Report.csv")
zipcode_df = pd.read_csv("Amazon_Sale_Report.csv")
coordinates_df = pd.read_csv("India_States-UTs.csv")

# Display sample data
zipcode_df.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [3]:
zipcode_df.drop(['Order ID', 'Date', 'Fulfilment', 'ship-service-level', 'Style', 'SKU', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-country', 'promotion-ids', 'B2B', 'fulfilled-by', 'Unnamed: 22', 'Style', 'SKU', 'Size', 'ASIN' ],axis=1, inplace=True)
zipcode_df.head()

,index,Status,Sales Channel,Category,Courier Status,Qty,ship-postal-code
0,0,Cancelled,Amazon.in,Set,NaN,0,400081.0
1,1,Shipped - Delivered to Buyer,Amazon.in,kurta,Shipped,1,560085.0
2,2,Shipped,Amazon.in,kurta,Shipped,1,410210.0
3,3,Cancelled,Amazon.in,Western Dress,NaN,0,605008.0
4,4,Shipped,Amazon.in,Top,Shipped,1,600073.0


In [4]:
zipcode_df_cleaned = zipcode_df.dropna(subset=['ship-postal-code']) #dropped the decimal in the zip code
zipcode_df_cleaned['ship-postal-code'] = zipcode_df_cleaned['ship-postal-code'].astype(int)
zipcode_df_cleaned.head()


,index,Status,Sales Channel,Category,Courier Status,Qty,ship-postal-code
0,0,Cancelled,Amazon.in,Set,NaN,0,400081
1,1,Shipped - Delivered to Buyer,Amazon.in,kurta,Shipped,1,560085
2,2,Shipped,Amazon.in,kurta,Shipped,1,410210
3,3,Cancelled,Amazon.in,Western Dress,NaN,0,605008
4,4,Shipped,Amazon.in,Top,Shipped,1,600073


--- 

### Bring in the API of India's zip code information

India has a total of 28 States and 8 Union Territories with at least 720 districts.<br>
The Indian postal departmenthas allotted a unique postal code of pin code to each<br>
district/village/town/city to ensure quick delivery of postal services.

In [5]:
# API base URL
url = "https://api.postalpincode.in/pincode/"
state_data = []

# Counters
# record_count = 1
# set_count = 1

print(zipcode_df_cleaned['ship-postal-code'][0]) #test
state_url = url + str(zipcode_df_cleaned['ship-postal-code'][0])
state_info = requests.get(state_url).json()
print(state_info[0]["PostOffice"][0]["State"]) #test


# Loop through all the zip codes to pull the state
for zipcode in zipcode_df_cleaned['ship-postal-code']:
    state_url = url + str(zipcode)
    try:
        state_info = requests.get(state_url).json()
        state_name = state_info[0]["PostOffice"][0]["State"]
        state_data.append(state_name)
    except: # need except for all try blocks
        print("Zip code not found. Skipping...")

zipcode_df_cleaned["State/UT"] = state_data
zipcode_df_cleaned.head()

400081
Maharashtra


,index,Status,Sales Channel,Category,Courier Status,Qty,ship-postal-code,State/UT
0,0,Cancelled,Amazon.in,Set,NaN,0,400081,Maharashtra
1,1,Shipped - Delivered to Buyer,Amazon.in,kurta,Shipped,1,560085,Karnataka
2,2,Shipped,Amazon.in,kurta,Shipped,1,410210,Maharashtra
3,3,Cancelled,Amazon.in,Western Dress,NaN,0,605008,Pondicherry
4,4,Shipped,Amazon.in,Top,Shipped,1,600073,Tamil Nadu


In [6]:
# Merging the latitude/longitude information zipcode_df_cleaned['State/UT']:
latlng_info_df = pd.merge(zipcode_df_cleaned, 
                          coordinates_df, 
                          how='outer', on='State/UT')
latlng_info_df.head()

,index,Status,Sales Channel,Category,Courier Status,Qty,ship-postal-code,State/UT,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andaman and Nicobar Islands,11.7401,92.6586
1,15.0,Shipped - Delivered to Buyer,Amazon.in,kurta,Shipped,1.0,515801.0,Andhra Pradesh,15.9129,79.7400
2,32.0,Shipped - Delivered to Buyer,Amazon.in,Set,Shipped,1.0,530016.0,Andhra Pradesh,15.9129,79.7400
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arunachal Pradesh,28.2180,94.7278
4,24.0,Shipped,Amazon.in,kurta,Shipped,1.0,784001.0,Assam,31.1048,77.1734


In [11]:
geo_latlng_info_df = latlng_info_df.rename(columns={'Latitude': 'Lat', 'Longitude': 'Lng'})
geo_latlng_info_df.head()

,index,Status,Sales Channel,Category,Courier Status,Qty,ship-postal-code,State/UT,Lat,Lng
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andaman and Nicobar Islands,11.7401,92.6586
1,15.0,Shipped - Delivered to Buyer,Amazon.in,kurta,Shipped,1.0,515801.0,Andhra Pradesh,15.9129,79.7400
2,32.0,Shipped - Delivered to Buyer,Amazon.in,Set,Shipped,1.0,530016.0,Andhra Pradesh,15.9129,79.7400
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arunachal Pradesh,28.2180,94.7278
4,24.0,Shipped,Amazon.in,kurta,Shipped,1.0,784001.0,Assam,31.1048,77.1734


In [9]:
# Output 
# sales_product_info_df.to_csv("../output/cleaned_Product Sales Info.csv", index=False)

---

### Trying to create a map that displays a point for every shipping postal code (destination).

In [17]:
%%capture --no-display

# Configure the map plot
shipped_state = geo_latlng_info_df["State/UT"]
map_plot_1 = geo_latlng_info_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 670,
    # frame_height = 500,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]